In [2]:
import os
import re
import time
import pandas as pd
import xarray as xr
from numpy import ones
from numpy.linalg import cholesky
from pandas_plink import read_plink1_bin
from limix.qc import quantile_gaussianize

In [3]:
import cellregmap 
cellregmap 

<module 'cellregmap' from '/hps/nobackup/stegle/users/acuomo/git_repos/CellRegMap/cellregmap/__init__.py'>

In [4]:
from cellregmap import run_interaction

In [5]:
input_files_dir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/new/input_files/"

In [6]:
## sample mapping file
## this file will map cells to donors 
## it will also only include donors we have single-cell data for (a subset of all of HipSci donors)
sample_mapping_file = input_files_dir+"sample_mapping_file.csv"
sample_mapping = pd.read_csv(sample_mapping_file, dtype={"genotype_individual_id": str, "phenotype_sample_id": str})

In [7]:
## genotype_individual_id are donor IDs, as found in the genotype matrix (G) and GRM covariance (K)
## phenotype_sample_id are cell IDs, as found in the scRNA-seq phenotype vector (y) and cell context covariance (C)
sample_mapping.head()

,genotype_individual_id,phenotype_sample_id
0,HPSI0114i-joxm_1,21843_1#10
1,HPSI0314i-fafq_1,21843_1#100
2,HPSI0314i-fafq_1,21843_1#101
3,HPSI1013i-wuye_2,21843_1#102
4,HPSI0114i-joxm_1,21843_1#103


In [8]:
## extract unique individuals
donors = sample_mapping["genotype_individual_id"].unique()
donors.sort()
print("Number of unique donors: {}".format(len(donors)))

Number of unique donors: 126


In [9]:
############################################
################ Kinship matrix ############
############################################

In [10]:
## read in GRM (genotype relationship matrix; kinship matrix)
kinship_folder="/hps/nobackup/hipsci/scratch/genotypes/imputed/2017-03-27/Full_Filtered_SNPs_Plink-F/"
kinship_file=kinship_folder+"hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.kinship"
K = pd.read_csv(kinship_file, sep="\t", index_col=0)
assert all(K.columns == K.index) #symmetric matrix, donors x donors

In [11]:
K = xr.DataArray(K.values, dims=["sample_0", "sample_1"], coords={"sample_0": K.columns, "sample_1": K.index})
K = K.sortby("sample_0").sortby("sample_1")
donors = sorted(set(list(K.sample_0.values)).intersection(donors))
print("Number of donors after kinship intersection: {}".format(len(donors)))

Number of donors after kinship intersection: 125


In [12]:
## subset to relevant donors
K = K.sel(sample_0=donors, sample_1=donors)
assert all(K.sample_0 == donors)
assert all(K.sample_1 == donors)

In [13]:
## and decompose such as K = hK @ hK.T (using Cholesky decomposition)
hK = cholesky(K.values)
hK = xr.DataArray(hK, dims=["sample", "col"], coords={"sample": K.sample_0.values})
assert all(hK.sample.values == K.sample_0.values)

In [14]:
del K
print("Sample mapping number of rows BEFORE intersection: {}".format(sample_mapping.shape[0]))
## subsample sample mapping file to donors in the kinship matrix
sample_mapping = sample_mapping[sample_mapping["genotype_individual_id"].isin(donors)]
print("Sample mapping number of rows AFTER intersection: {}".format(sample_mapping.shape[0]))

Sample mapping number of rows BEFORE intersection: 34256
Sample mapping number of rows AFTER intersection: 33964


In [15]:
############################################
##### expand from donors to cells ##########

In [16]:
## use sel from xarray to expand hK (using the sample mapping file)
hK_expanded = hK.sel(sample=sample_mapping["genotype_individual_id"].values)
assert all(hK_expanded.sample.values == sample_mapping["genotype_individual_id"].values)

In [17]:
######################################
############### Genotypes ############
######################################

In [18]:
## read in genotype file (plink format)
plink_folder = "/hps/nobackup/hipsci/scratch/genotypes/imputed/2017-03-27/Full_Filtered_SNPs_Plink/"
plink_file = plink_folder+"hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.bed"
G = read_plink1_bin(plink_file)

Mapping files: 100%|██████████| 3/3 [06:01<00:00, 120.48s/it]


In [19]:
######################################
########## Cell contexts #############
######################################

In [20]:
# cells by PCs (20)
C_file = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/new/input_files/20PCs.csv"
C = pd.read_csv(C_file, index_col = 0)
C = xr.DataArray(C.values, dims=["cell", "pc"], coords={"cell": C.index.values, "pc": C.columns.values})
C = C.sel(cell=sample_mapping["phenotype_sample_id"].values)
assert all(C.cell.values == sample_mapping["phenotype_sample_id"].values)

In [21]:
# quantile normalise cell contexts
C_gauss = quantile_gaussianize(C)

In [22]:
#####################################
############ Phenotypes #############
#####################################

In [23]:
# Phenotype (single-cell expression)
phenotype_file = input_files_dir+"phenotype.csv.pkl"
phenotype = pd.read_pickle(phenotype_file)
print("Phenotype shape BEFORE selection: {}".format(phenotype.shape))
phenotype = xr.DataArray(phenotype.values, dims=["trait", "cell"], coords={"trait": phenotype.index.values, "cell": phenotype.columns.values})
phenotype = phenotype.sel(cell=sample_mapping["phenotype_sample_id"].values)
print("Phenotype shape AFTER selection: {}".format(phenotype.shape))
assert all(phenotype.cell.values == sample_mapping["phenotype_sample_id"].values)

Phenotype shape BEFORE selection: (11231, 34256)
Phenotype shape AFTER selection: (11231, 33964)


In [24]:
#####################################
############ Filter file ############
#####################################

In [25]:
# filter file (columns: snp_id, gene)
endo_eqtl_file = input_files_dir+"endodiff_eqtl_allconditions_FDR10pct.csv"
endo_eqtl = pd.read_csv(endo_eqtl_file, index_col = False)
endo_eqtl["chrom"] = [int(i[:i.find("_")]) for i in endo_eqtl["snp_id"]]
endo_eqtl.head(2)

,snp_id,feature,stage,chrom
0,5_149826526_C_T,ENSG00000164587_RPS14,ips,5
1,11_57283988_C_T,ENSG00000134809_TIMM10,ips,11


In [175]:
chrom = 15
# and consider eGenes on that chromosome
genes = endo_eqtl[endo_eqtl['chrom']==int(chrom)]['feature'].unique()

In [176]:
len(genes)

106

In [177]:
outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/CRM_int_PCA/"

In [178]:
genes

array(['ENSG00000140319_SRP14', 'ENSG00000175265_GOLGA8A',
       'ENSG00000140443_IGF1R', 'ENSG00000136371_MTHFS',
       'ENSG00000225151_GOLGA2P7', 'ENSG00000197696_NMB',
       'ENSG00000172345_STARD5', 'ENSG00000128833_MYO5C',
       'ENSG00000182511_FES', 'ENSG00000166946_CCNDBP1',
       'ENSG00000068793_CYFIP1', 'ENSG00000140545_MFGE8',
       'ENSG00000250021_C15orf38-AP3S2', 'ENSG00000157823_AP3S2',
       'ENSG00000185043_CIB1', 'ENSG00000128891_C15orf57',
       'ENSG00000041357_PSMA4', 'ENSG00000140451_PIF1',
       'ENSG00000090487_SPG21', 'ENSG00000178802_MPI',
       'ENSG00000166938_DIS3L', 'ENSG00000137822_TUBGCP4',
       'ENSG00000215252_GOLGA8B', 'ENSG00000128849_CGNL1',
       'ENSG00000103876_FAH', 'ENSG00000260916_CCPG1',
       'ENSG00000092531_SNAP23', 'ENSG00000137812_CASC5',
       'ENSG00000168803_ADAL', 'ENSG00000069667_RORA',
       'ENSG00000198794_SCAMP5', 'ENSG00000140400_MAN2C1',
       'ENSG00000103723_AP3B2', 'ENSG00000128829_EIF2AK4',
       'ENSG0

In [179]:
for gene_name in genes:
    trait_name = re.sub("_.*","",gene_name)
    outfilename = outdir + str(gene_name) + ".tsv"
    if os.path.exists(outfilename):
        print("File already exists, exiting")
        continue
    leads = endo_eqtl[endo_eqtl['feature']==gene_name]['snp_id'].unique()
    G_sel = G[:,G['snp'].isin(leads)]
    G_expanded = G_sel.sel(sample=sample_mapping["genotype_individual_id"].values)
    assert all(hK_expanded.sample.values == G_expanded.sample.values)
#     trait_name = re.sub("-",".",trait_name)
    y = phenotype.sel(trait=gene_name)
    y = quantile_gaussianize(y)
    y = y.values.reshape(y.shape[0],1)
    n_cells = phenotype.shape[1]
    W = ones((n_cells, 1))
    GG = G_expanded.values
    print("Running for gene {}".format(gene_name))
    pvals = run_interaction(y=y, W=W, E=C_gauss.values[:,0:10], E1=C_gauss.values[:,0:10], E2=C.values[:,0:20], G=GG, hK=hK_expanded)[0]
    pv = pd.DataFrame({"chrom":G_expanded.chrom.values,
           "pv":pvals,
           "variant":G_expanded.snp.values})
    pv.to_csv(outfilename)

File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already exists, exiting
File already e